# Meter Inventory Transformation for Import

### This notebook is a self-contained set of code and instructions that will enable you to convert individual files or batches of files from the vendor-provided format to the Milsoft-import format with minimal effort

## Select Files

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from io import BytesIO
from IPython.display import display
import zipfile
import shutil
import base64
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Folder to save transformed output
output_folder = "voila_output"
os.makedirs(output_folder, exist_ok=True)

SIZE_TO_CODE = {
    '3"': "300",
    '5/8"': "058",
    '1"': "100",
    '1-1/2"': "150",
    '2"': "200",
    '4"': "400",
    '6"': "600"
}

output_folder = "test_out"
os.makedirs(output_folder, exist_ok=True)

file_uploader = widgets.FileUpload(
    accept = '.xlsx',
    multiple = True,
    description = "Upload Excel files"
)


# Button to trigger processing
process_button = widgets.Button(
    description="Transform Files",
    button_style='success',
    icon='cogs'
)

download_button = widgets.Button(
    description="Download All CSVs as ZIP",
    button_style='info',
    icon='download'
)



# Output widget for logs
log_output = widgets.Output()

display(file_uploader, process_button, download_button, log_output)

# This is the HTML widget you'll update after generating the file
download_link_widget = widgets.HTML(value="")
display(download_link_widget)

def edit_download_html(htmlWidget, filename, title="Click to download: "):
    htmlWidget.value = "<i class=\"fa fa-spinner fa-spin fa-2x fa-fw\"></i><span class=\"sr-only\">Loading...</span>"
    with open(filename, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    html = f'<a download="{filename}" href="data:application/zip;base64,{b64}" target="_blank">{title}{filename}</a>'
    htmlWidget.value = html


FileUpload(value=(), accept='.xlsx', description='Upload Excel files', multiple=True)

Button(button_style='success', description='Transform Files', icon='cogs', style=ButtonStyle())

Button(button_style='info', description='Download All CSVs as ZIP', icon='download', style=ButtonStyle())

Output()

In [ ]:
def map_meter_size(size_str):
    if pd.isna(size_str):
        return ""
    for key, val in SIZE_TO_CODE.items():
        if key in size_str:
            return val
    return ""

def transform_excel_to_csv(df: pd.DataFrame, output_path: str):

    # Create transformed DataFrame
    output = pd.DataFrame()
    output["MISER"] = df["Meter SN"].fillna(df["Register SN"].astype(str))
    output["MISERV"] = 3
    output["MITSTDT"] = df["Meter Test Date"].fillna(df["Ship Date"])
    output["MIMTYPE"] = "Sensus"
    output["MIPDTE"] = "01/01/0001"
    output["MIMULT"] = 1
    output["MIKIND"] = "W"
    output["MIDIAL"] = 9
    output["MICDTE"] = "01/01/0001"
    output["MIMTRS"] = "IPK"
    output["MIHANDHELD"] = "N"
    output["MIOPEN2"] = df["Meter Size"].apply(map_meter_size)

    # Fill in missing required columns with blanks
    TARGET_COLUMNS = [
        'MISER', 'MISERV', 'MITSTDT', 'MITSTR', 'MIASFKWH', 'MIASLKWH', 'MIASFFL', 'MIASFLL',
        'MIASLLL', 'MIASLFL', 'MIWHY', 'MIMOD#', 'MIMTYPE', 'MIVOU', 'MIPDTE', 'MIPCST',
        'MIMULT', 'MIMNO', 'MIMFG', 'MITYPE', 'MIKIND', 'MIPHSE', 'MIVOLT', 'MIAMPS',
        'MIWIRE', 'MIRACTION', 'MIDIAL', 'MIFORM', 'MIRR', 'MICLAS', 'MIPF', 'MIOSER',
        'MIOREAD', 'MIOKWRD', 'MICDTE', 'MIWHSE', 'MIMCT', 'MIMTRS', 'MIOPEN2', 'MIOPEN3',
        'MIHANDHELD'
    ]

    for col in TARGET_COLUMNS:
        if col not in output.columns:
            output[col] = ""

    # Reorder the columns to match the required format
    output = output[TARGET_COLUMNS]

    # Save to CSV
    output.to_csv(output_path, index=False, encoding="utf-8")
    print(f"✅ Output saved to: {output_path}")
    
    
def create_zip_of_outputs(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

In [ ]:
def on_process_clicked(b):
    shutil.rmtree(output_folder)
    os.makedirs(output_folder)
    log_output.clear_output()
    with log_output:
        if not file_uploader.value:
            print("⚠️ Please upload at least one file before processing.")
            return

        for file_info in file_uploader.value:
            filename = file_info['name']
            if not filename.endswith(".xlsx"):
                print(f"❌ Skipping non-Excel file: {filename}")
                continue
            
            print(f"🔄 Processing: {filename}")
            file_bytes = BytesIO(file_info['content'])

            try:
                df = pd.read_excel(file_bytes)
                output_filename = os.path.splitext(filename)[0] + "_transformed.csv"
                output_path = os.path.join(output_folder, output_filename)
                transform_excel_to_csv(df, output_path)
                print(f"✅ Saved: {output_path}")
            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")
                
def on_download_clicked(b):
    log_output.clear_output()
    with log_output:
        zip_path = "transformed_files.zip"
        create_zip_of_outputs(output_folder, zip_path)
        print("📦 ZIP file created.")
        edit_download_html(download_link_widget, zip_path, title="👉 Click to download: ")

download_button.on_click(on_download_clicked)

process_button.on_click(on_process_clicked)

'\nfor index, file_info in enumerate(file_uploader.value):\n    filename = file_info[\'name\']\n    if not filename.endswith(".xlsx"):\n        print(f"❌ Skipping non-Excel file: {filename}")\n        continue\n\n    print(f"🔄 Processing: {filename}")\n    file_bytes = BytesIO(file_info[\'content\'])\n\n    try:\n        df = pd.read_excel(file_bytes)\n        output_filename = os.path.splitext(filename)[0] + "_transformed"\n        output_path = os.path.join(output_folder, output_filename + ".csv")\n        transform_excel_to_csv(df, output_path)\n    except Exception as e:\n        print(f"❌ Failed to process {filename}: {e}")'